<a href="https://colab.research.google.com/github/AttraqtiA/Cat-Dog-ImageProcessing/blob/main/Week3_CatDog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

rootPath = "/content/drive/MyDrive/Semester5_DSU/Advanced_AI/CatDogDataset"


In [2]:
imageGenerator = ImageDataGenerator(rescale=1./255,
                                    rotation_range=20,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    brightness_range=[.2,.2],
                                    horizontal_flip=True,
                                    validation_split=.2)

# We're going to split it into about 80%  train and 20% test

In [3]:
trainGen = imageGenerator.flow_from_directory(os.path.join(rootPath,'training_set'),
            target_size=(64,64),
            class_mode='binary', # ADD FOR BINARY
            subset='training')
validationGen = imageGenerator.flow_from_directory(os.path.join(rootPath,'training_set'),
            target_size=(64,64),
            class_mode='binary',
            subset='validation')

Found 6404 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.


In [4]:
model = Sequential()
model.add(layers.InputLayer(input_shape=(64,64,3)))
model.add(layers.Conv2D(16,(3,3),(1,1),'same',activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(rate=0.5))

model.add(layers.Conv2D(32, (3, 3), (1, 1), 'same', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(rate=0.5))

model.add(layers.Conv2D(64, (3, 3), (1, 1), 'same', activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(rate=0.5))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

# Option 1: Single output neuron + sigmoid activation + binary_crossentropy loss for binary classification.
# Option 2: Two output neurons + softmax activation + categorical_crossentropy loss for categorical classification.

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 64, 64, 16)          │             448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 32, 32, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 32, 32, 16)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 32, 32, 32)          │           4,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16, 16, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 16, 16, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 8, 8, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 4096)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │         262,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 287,905 (1.10 MB)

 Trainable params: 287,905 (1.10 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
# Create a program to read data from Kaggle

from tensorflow.keras import optimizers
from keras.callbacks import EarlyStopping

opt = optimizers.Adam(learning_rate=0.001)

model.compile(
    optimizer=opt,
    loss='binary_crossentropy',
    metrics=['acc']
)

In [ ]:
epochs = 200
es = EarlyStopping(monitor='val_loss', mode='min', patience=5)

history = model.fit(
    trainGen,
    epochs=epochs,
    steps_per_epoch=trainGen.samples // epochs,
    validation_data=validationGen,
    validation_steps=trainGen.samples // epochs,
    callbacks=[es]
)

Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


32/32 ━━━━━━━━━━━━━━━━━━━━ 929s 24s/step - acc: 0.4637 - loss: 0.6971 - val_acc: 0.4941 - val_loss: 0.6933
Epoch 2/200
 8/32 ━━━━━━━━━━━━━━━━━━━━ 2:25 6s/step - acc: 0.5423 - loss: 0.6923

In [ ]:
# matplotlib visualization

import matplotlib.pyplot as plt
from pylab import figure, axes, pie, title, savefig

def show_graph(history_dict):
  accuracy = history_dict['acc']
  val_accuracy = history_dict['val_acc']
  loss = history_dict['loss']
  val_loss = history_dict['val_loss']
  epochs = range(1, len(loss) + 1)
  plt.figure(figsize=(16, 2))
  plt.subplot(121)
  plt.subplots_adjust(top=2)
  plt.plot(epochs, accuracy, label='Training accuracy')
  plt.plot(epochs, val_accuracy, label='Validation accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
      fancybox=True, shadow=True, ncol=5)
# plt.legend(bbox_to_anchor=(1, -0.1))

  plt.subplot(122)
  plt.plot(epochs, loss, label='Training loss')
  plt.plot(epochs, val_loss, label='Validation loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
  fancybox=True, shadow=True, ncol=5)
# plt.legend(bbox_to_anchor=(1, 0))

  fig = plt.gcf()
  plt.show()

show_graph(history.history)

In [ ]:
testGenerator = ImageDataGenerator(
  rescale=1./255
)

testGen = imageGenerator.flow_from_directory(
  os.path.join(rootPath, 'test_set'),
  target_size=(64, 64),
  class_mode='binary'
)

model.evaluate(testGen)

In [ ]:
from tensorflow.keras.preprocessing.image import array_to_img
import numpy as np

cls_index = ['CAT', 'DOG']
imgs = testGen.next()
arr = imgs[0][0]
img = array_to_img(arr).resize((128, 128))
plt.imshow(img)
result = model.predict(arr.reshape(1, 64, 64, 3))

# Find the class index with the highest probability
predicted_class = np.argmax(result[0])
print('Prediction: {}'.format(cls_index[predicted_class]))
print('Answer: {}'.format(cls_index[np.argmax(imgs[1][0])]))